In [2]:
import json
import requests
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import pandas as pd

## Code to convert Esri FeatureSets to GeoJSON

In [3]:
def process_features(features):
    # Convert 'rings' into shapely Polygon/MultiPolygon objects
    def convert_geometry(rings):
        if len(rings) == 1:
            return Polygon(rings[0])
        else:
            # Assuming each 'ring' in 'rings' should be a separate polygon
            return MultiPolygon([Polygon(ring) for ring in rings])

    for feature in features:
        feature['geometry'] = convert_geometry(feature['geometry']['rings'])
        feature['properties'] = feature.pop('attributes')  # rename 'attributes' to 'properties'

    return features

## Visualize Area of interest

In [ ]:
with open("US_PAD/tests/AOI_Yosemite.json") as aoi:
    aoi = json.load(aoi)

features = process_features(aoi["features"]) # convert Esri FeatureSet to shapely geometry objects

gdf = gpd.GeoDataFrame.from_features(aoi["features"])
gdf.crs = "EPSG:4326"
gdf.explore()

## Query Intersect Endpoint and Display intersecting protected areas

In [ ]:
with open("US_PAD/tests/AOI_Yosemite.json") as aoi_file:
    aoi = json.load(aoi_file)

data = {"aoi": aoi}
response = requests.post("http://localhost:8000/padApi/aoi-intersect", data=json.dumps(data))
featureset_features_list = response.json()
intersected_features_shapely = process_features(featureset_features_list)

gdf = gpd.GeoDataFrame.from_features(intersected_features_shapely)
gdf.crs = "EPSG:4326"
gdf.explore()

## Query Overlap Endpoint and Display results in a table

In [10]:
with open('US_PAD/tests/AOI_Yosemite.json') as f:
    aoi = json.load(f)

data = {"aoi": aoi, "intersected_features": featureset_features_list}
response = requests.post("http://localhost:8000/padApi/aoi-overlap", data=json.dumps(data))

# Process the response
data = response.json()
df = pd.DataFrame(data)
display(df)

,Yosemite National Park
FED,100
